# Project Group 11

Members:  Alessandro Casati (6544649),  Bas van den Muijsenberg (5797578) , Hylke Bleeker (5589355), Jan-Pieter Vermeer (6340261),  Mike Geerts (6276210)


# Research Objective

## Analysis and delay prediction of flights across top 20 airports in Europe in 2023-24

**Research Question**  
What factors drive airport delays across the top 10 European airports in 2023-24, and can we build predictive models to identify possible delays for a flight using the information available?

**Sub Questions**
- What type of delays are occurring most at the airports?
- Which airports experience the highest, the lowest delay, and most consistent delay rates?
- How do delays relate to traffic volume and efficiency?
- How do delays distribute geographically across Europe, and which airports stand out as persistent “delay hotspots”?
- Which features (traffic levels, weekday/weekend, month, etc.) are the strongest predictors of high-delay days?
- What model can predict the chance of delay and the amount delay for a flight?

# Contribution Statement

*Be specific. Some of the tasks can be coding (expect everyone to do this), background research, conceptualisation, visualisation, data analysis, data modelling*

**Author 1**:

**Author 2**:

**Author 3**:

# Data Used

## Datasets
| Dataset | Dataset description (function) | Columns needed |
|---|---|---|
| Airport Traffic | Airport capacity of inbound and outbound flights for network manager and airport operator | All |
| Airport ATFM_Delay | The Airport Arrival ATFM Delay provides an indication of ATFM delays on the ground due to constraints at airports | FLT_DATE, APT_ICAO (to merge DB) + all the remaining types of delays |
| Airport Punctuality | Daily performance tracker of airport punctuality, delays, and operational efficiency | All + add APT_ICAO for merging |
| All_Pre_Departure_Delays | Daily overview of total pre-departure delay in minutes per airport that includes all delay causes | All |
| ATC_Pre_Departure_Delay | Proxy for ATC induced delays at the departure stand (IATA delay code 89) | Pending (depends on data analysis findings) |
| Additional taxi-out time | Taxi time from gate to runway and associated delay | All |
| Additional taxi-in time | Taxi time from runway to gate and associated delay | All |

## Information extracted from datasets
| Dataset | Key Information |
|---|---|
| Airport Traffic | IFR arrivals, departures for Airport Operator and Network Manager |
| Airport ATFM_Delay | Disruption type and count, number of delayed arrival flights |
| Airport Punctuality | Arrival & departure punctuality %, avg delay (dep & arr), operated schedules % |
| Pre_Departure_Delay | AO departures, AO pre-departure delay (minutes) |
| Additional taxi-out time | Number of flights with available data + total taxi in time |
| Additional taxi-in time | Number of flights with available data + total taxi out time |

# Data Pipeline

## Data Exploration & Cleaning
- Import & clean the datasets: remove missing/invalid values, select only useful columns and merge.
- Create basic delay metrics:
  - Delay categories (On time, Delayed, Delayed15)
  - Day category (Weekend, Weekdays)

## Descriptive Data Analysis
- Time patterns: heatmaps/line charts/stacked charts (delays by day, month, cause).
- Geospatial view: interactive maps color-coded by average airport delay.
- Efficiency scatterplots: traffic volume vs. % delay

## Predictive Modeling
- Predict delay occurrence for a flight using departure airport and its performances, average airport delay, weekday/weekend & month.
- Models: Logistic Regression and/or Random Forest.
- Testing with 2025 data (unseen during training).

## Clustering & Patterns
- Cluster airports by daily delay patterns (e.g., “always congested,” “seasonal peaks,” “mostly smooth”).

# Data Cleaning

In [16]:
import pandas as pd
import numpy as np


Finding the top 10 airports by flow size

In [7]:
df_AT = pd.read_excel("datasets/AirportTraffic.xlsx")


In [59]:
df_top10 = df_AT.groupby("APT_ICAO")["FLT_TOT_1"].sum().sort_values(ascending=False).head(20).reset_index() #Change "20" to change the number of airports analysed
# Step 2: Merge with original data (to bring other info)
df_top10 = df_top10.merge(df_AT[["APT_ICAO", "APT_NAME", "STATE_NAME"]].drop_duplicates(), on="APT_ICAO", how="left")
df_top10.head(20)

,APT_ICAO,FLT_TOT_1,APT_NAME,STATE_NAME
0,LTFM,1014869,Istanbul,Türkiye
1,EHAM,947254,Amsterdam - Schiphol,Netherlands
2,EGLL,933098,London - Heathrow,United Kingdom
3,LFPG,921916,Paris-Charles-de-Gaulle,France
4,EDDF,871219,Frankfurt,Germany
5,LEMD,809535,Madrid - Barajas,Spain
6,LEBL,666969,Barcelona,Spain
7,EDDM,624316,Munich,Germany
8,LIRF,582158,Rome - Fiumicino,Italy
9,EGKK,522331,London - Gatwick,United Kingdom


## TO-DO list, by Wed 8/10
| Dataset | Key Information | Assigned to |
|---|---|---|
| Airport Traffic | Imported + top20 defined: missing total number of departures and arrivals | Ale |
| Airport ATFM_Delay | Extract top20 aiports by comparing it with df_top10 and overall and sum for all the 2 years | Bas |
| Airport Punctuality | Filter for 23-24, find top 20, compare with df_top20 (some airports might be missing), get all the columns | Mike |
| Pre_Departure_Delay | TBD. is the data actually useful for our RQ, if yes extract top20 + last 5 columns | JP |
| Additional taxi-out time | Extract top20 airports + TF, TOT_REF_NB_FL, TOT_REF&ADD_TIME_MIN, PIVOT_LABEL | Hylke |
| Additional taxi-in time | Extract top20 airports + TF, TOT_REF_NB_FL, TOT_REF&ADD_TIME_MIN, PIVOT_LABEL | Hylke |

Some comments: for now we focus on the totals for 2023-24 (combined). Watch out that some datasets don't have matching columns and you can't merge them together, you might need to compare them manually or use other references (especially for Punctuality dataset)